In [1]:
import os
import pandas as pd
import pymysql
import numpy as np

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 연도별 식당 업종 및 업태 분석 정보

# RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_2024_06

### https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=e573bd20-2641-11ec-bbc0-d7035fffebeb

In [5]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_2024_06'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [3]:
with dev_db_conn() as conn:
    sql = """
    SELECT CONCAT(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), ' ', T1.AREA_NM_2) AS 'NAME',
     LEFT(T1.BSNS_LCNC_NO, 4) AS 'YEAR',
     SUM(CASE WHEN T3.LABEL_NM = '한식' THEN 1 ELSE 0 END) AS 'KO',
     SUM(CASE WHEN T3.LABEL_NM = '양식' THEN 1 ELSE 0 END) AS 'WE',
     SUM(CASE WHEN T3.LABEL_NM = '중식' THEN 1 ELSE 0 END) AS 'CH',
     SUM(CASE WHEN T3.LABEL_NM = '일식' THEN 1 ELSE 0 END) AS 'JP',
     SUM(CASE WHEN T3.LABEL_NM = '카페' THEN 1 ELSE 0 END) AS 'CA',
     #SUM(CASE WHEN T3.LABEL_NM = '베이커리' THEN 1 ELSE 0 END) AS 'BA',
     SUM(CASE WHEN T1.RSTR_NM LIKE '%휴게소%' THEN 1 ELSE 0 END) AS 'RSTAREA_CO',
     SUM(CASE WHEN T1.BSNS_STATM_BZCND_NM IN ('정종/대포집/소주방', '호프/통닭') THEN 1 ELSE 0 END) AS 'PUB_CO',
     SUM(CASE WHEN T3.LABEL_NM = '기타' THEN 1 ELSE 0 END) AS 'ETC'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_LABEL T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_LABEL_MASTER T3 ON T2.LABEL_MASTER_ID = T3.ID
    WHERE T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_2 IS NOT NULL AND T1.BSNS_LCNC_NO IS NOT NULL
    AND T1.BSNS_LCNC_NO NOT LIKE 'kios%' AND T1.BSNS_LCNC_NO NOT LIKE 'nave%' AND T1.BSNS_LCNC_NO NOT LIKE 'publ%'
    GROUP BY NAME, YEAR
    ORDER BY NAME ASC, YEAR ASC;
    """
    df = pd.read_sql(sql, conn)
    
# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

# id 값 채우기
df['index'] = df.index + 1

In [6]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT INTO redtable_bigdata_center.RT_YEAR_ACCTO_RSTRNT_INDUTY_ND_BZCND_ANALS_INFO_2024_06({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["index"], row["NAME"], row["YEAR"], row["KO"], row["WE"],
                                  row["CH"], row["JP"], row["CA"], row["RSTAREA_CO"], row["PUB_CO"],
                                  row["ETC"]), axis=1).tolist()
        curs.executemany(sql, val)